In [1]:
#!/usr/bin/python3

# PURPOSE: to generate the vars.txt file (which is a subject x Subject Measures matrix) used by Smith et al. in their analysis
# see here for reference: https://www.fmrib.ox.ac.uk/datasets/HCP-CCA/

# USAGE: multiple files are needed:
# 1. a .txt file containing the names of the subject measures (SMs) to be used in the analysis
# 2. a .txt file containing the names of all subjects to be analyzed (their subject IDs)
# 3. the behavioral data from HCP
# 4. the 'restricted' data from HCP (requires special access, must request this)
# 5. the rfMRI_motion.txt file
# 6. the quarter/release info file (named varsQconf.txt)

# EXAMPLE USAGE ON CMD LINE:
# ./generate_vars.py column_headers.txt subjects.txt <behavioral data> <restricted data> rfMRI_motion.txt varsQconf.txt

import numpy as np
import pandas as pd
from pandas import DataFrame
from numpy import genfromtxt
import os
import sys
from pprint import pprint

cwd = os.getcwd()

column_headers_file = os.path.join(cwd, 'column_headers.txt')
filenames_file = os.path.join(cwd, 'filenames.txt')
behavioral_data_file = os.path.join(cwd, 'unrestricted_goyaln_8_29_2019_13_25_48.csv')
restricted_data_file = os.path.join(cwd, 'RESTRICTED_athomas24_12_18_2018_12_27_32.csv')
rfMRI_data_file = os.path.join(cwd, 'rfMRI_motion.txt')
varsQconf_file = os.path.join(cwd, 'varsQconf.txt')

# get the column headers, and names of subjects
column_headers = [line.rstrip('\n') for line in open(os.path.join(cwd,column_headers_file))]
subjects = [line.rstrip('.pconn.nii\n') for line in open(os.path.join(cwd,filenames_file))]

# now import "behavioral" and "restricted" datasets into Pandas dataframes
# note that these have data on many more subjects than the 461 used in the Smith et al study (these have all subjects in the HCP_1200 release)
behavioral_data = pd.read_csv(os.path.join(cwd, behavioral_data_file))
restricted_data = pd.read_csv(os.path.join(cwd, restricted_data_file))


# Now we will filter out only the rows that correspond to the subjects specified in subjects.txt
# Sanity check, making sure that the filtering occurs correctly
print('behavior shape before', behavioral_data.shape)
print('shape of restricted before', restricted_data.shape)

#filter the behavioral and restricted datasets to contain only the relevant 461 subject data
behavioral_data = behavioral_data[behavioral_data['Subject'].isin(subjects)]
restricted_data = restricted_data[restricted_data['Subject'].isin(subjects)]

print('behavior shape after', behavioral_data.shape)
print('shape of restricted after', restricted_data.shape)

behavior shape before (1206, 582)
shape of restricted before (1206, 201)
behavior shape after (460, 582)
shape of restricted after (460, 201)


In [2]:
# Now import the rfMRI and quarter/release (varsQconf) data
varsqconf = pd.read_csv(varsQconf_file, names=['quarter/release'])
rfmri = pd.read_csv(rfMRI_data_file, sep=" ", names=['rfmri_motion'])

In [3]:
# reindex so that the varsqconf has the correct subject IDs as its row labels
varsqconf.index = rfmri.index

In [4]:
# concatenate the rfMRI and varsQconf data (we will need to do this later anyway)
rfmri_varsqconf = pd.concat([rfmri, varsqconf], axis=1)

In [5]:
# There appears to be one subject missing from the original list of 461 (for whom we have partial correlation netmats)

In [6]:
behav_sub_list = list(behavioral_data['Subject'])
restrict_sub_list = list(restricted_data['Subject'])

In [7]:
# find the missing subject in each case
np.setdiff1d(subjects,behav_sub_list)

array(['142626'], dtype='<U6')

In [8]:
np.setdiff1d(subjects,restrict_sub_list)

array(['142626'], dtype='<U6')

In [9]:
# After investigating, subject 142626 was actually a duplicate and was removed from the ConnectomeDB
# as a result, this analysis will need to use the subset of 460 subjects

# From HCP: https://www.humanconnectome.org/study/hcp-young-adult/document/900-subjects-data-release
# "IMPORTANT: Subject 142626 removed from ConnectomeDB.
# We have recently found that subject 142626, released in the 500 Subjects Release (June 2014), 
# has the same identity as another subject in the HCP study. Thus, we have removed all data for 
# subject 142626 from ConnectomeDB. For any ongoing analyses, we recommend that if possible you 
# exclude subject 142626 from your analyses."

In [10]:
# Now let's remove sub 142626 from our list of subjects, so that it matches the subjects in the now filtered behavioral_data and restricted_data dataframes
subjects.remove('142626')

# Also, lets drop the subject 142626 from the rfmri_varsqconf dataframe
rfmri_varsqconf = rfmri_varsqconf.drop(index=142626)

In [11]:
# get the names of column headers
behav_headers=list(behavioral_data.columns.values)
restrict_headers=list(restricted_data.columns.values)

# Make lowercase
column_headers=[element.lower() for element in column_headers]
behav_headers=[element.lower() for element in behav_headers]
restrict_headers=[element.lower() for element in restrict_headers]

In [13]:
missing_in_behav = np.setdiff1d(column_headers,behav_headers)
missing_in_restrict = np.setdiff1d(column_headers,restrict_headers)

In [14]:
missing_in_behav_and_restrict = np.setdiff1d(missing_in_behav,restrict_headers)

In [15]:
# Although these column headers are missing from our behavioral and restricted datasets, we will proceed to generate the vars.txt matric anyway
# Note that in Smith et al, these empty columns were included in the matrix fed into the CCA
# This resulted in a 461 × 158 matrix S4 (which still included some missing data). These 158 SMs fed into the CCA are now listed using their formal database naming

In [16]:
# Now lets fetch the relevant columns from each df

# first, check for overlap between behavioral and restricted data
len(np.setdiff1d(behav_headers,restrict_headers))

581

In [17]:
# It looks like aside from the subject id, there is no overlap between columns.

In [18]:
# First, lets get the column names that are overlapped in each
overlap_in_behav = np.intersect1d(column_headers,behav_headers)
overlap_in_restrict = np.intersect1d(column_headers,restrict_headers)

# Sanity check, confirm that the overlaps are contained by the arrays (aka check for differences)
print(np.setdiff1d(overlap_in_behav, behav_headers))
print(np.setdiff1d(overlap_in_restrict, restrict_headers))

[]
[]


In [19]:
# Okay, so we found that there is no overlaps between the column_headers and the behavior/restricted datasets which will be used to construct vars.txt

# now lets do some simple math to make sure everything adds up
total = len(overlap_in_behav) - 1 + len(overlap_in_restrict) #-1 to account for double count of 'subject'
print(total)

461


In [1]:
len(column_headers) - total
len(missing_in_behav_and_restrict) 

NameError: name 'column_headers' is not defined

In [21]:
# Now pull out the columns and their data
# first we will need to convert all the column headers to lowercase
behavioral_data.columns = behavioral_data.columns.str.lower()
restricted_data.columns = restricted_data.columns.str.lower()

behavioral_data_filtered_cols = behavioral_data[overlap_in_behav]
restricted_data_filtered_cols = restricted_data[overlap_in_restrict]

In [34]:
# check that all dimensions are correct before we attempt to concat the dataframes
print(behavioral_data_filtered_cols.shape)
print(restricted_data_filtered_cols.shape)
print(rfmri_varsqconf.shape)

(460, 285)
(460, 177)
(460, 2)


In [35]:
# concat the dataframes

# first reindex all of them to match rfmri_varsqconf
behavioral_data_filtered_cols.index = rfmri_varsqconf.index
restricted_data_filtered_cols.index = rfmri_varsqconf.index

vars = pd.concat([behavioral_data_filtered_cols, restricted_data_filtered_cols, rfmri_varsqconf], axis = 1)

In [36]:
vars = vars.drop(columns='subject')
vars = vars.reindex(columns = column_headers)

In [33]:
vars.to_csv("vars.txt")